# <div align="center" style="color: #ff5733;">New KPI Report</div>

## *Requestor is Jennyfer & Anne*

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
# path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

c:\Users\DwaipayanChakroborti\Myenv\reporting\lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


# FPD30 November 2024 Month End

In [3]:
sq = """SELECT t1.new_loan_type,
case when sum(t1.obsFPD30) = 0 then 0 else ROUND(SUM(CASE when t1.defFPD30 = 1 then 1 ELSE 0 end )/sum(t1.obsFPD30)*100,3) end as percentage_of_FPD30
from `prj-prod-dataplatform.risk_credit_mis.loan_master_table` t1
inner JOIN prj-prod-dataplatform.risk_credit_mis.loan_bucket_flow_report_core t2
ON
t1.loanAccountNumber = t2.loanAccountNumber
WHERE t1.flagDisbursement = 1
and t1.new_loan_type in ('Quick','Flex','SIL-Instore', 'SIL ZERO', 'SIL Competitor', 'SIL Repeat')
and t2.loanStatus in ('In Arrears','Normal')
and t2.bucketDate in ('2024-11-30')
GROUP BY 1;"""

nov24fpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')
nov24fpd30

Job ID 3cfdb708-e5b2-4ab1-8e72-859b61144187 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,new_loan_type,percentage_of_FPD30
0,Quick,13.853
1,SIL Competitor,0.000
2,SIL ZERO,8.960
3,Flex,0.222
4,SIL Repeat,0.000
5,SIL-Instore,8.092


# 30+, 60+, 90+ for November 2024

In [4]:
sq = """with b as
(select t1.new_loan_type 
, count(distinct t2.loanAccountNumber) cnt_loan
, count(distinct case when t2.Max_current_DPD >=30 then t2.loanAccountNumber end) thirtyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >=30 then t2.loanAccountNumber end) obsthirtyplus
, count(distinct case when t2.Max_current_DPD >=60 then t2.loanAccountNumber end) sixtyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >=60 then t2.loanAccountNumber end) obssixtyplus
, count(distinct case when t2.Max_current_DPD >=90 then t2.loanAccountNumber end) nintyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >= 90 then t2.loanAccountNumber end) obsnintyplus
from `risk_credit_mis.loan_bucket_flow_report_core` t2
inner join `risk_credit_mis.loan_master_table` t1 on t2.loanAccountNumber = t1.loanAccountNumber
WHERE t1.flagDisbursement = 1
and t1.new_loan_type in ('Quick','Flex','SIL-Instore', 'SIL ZERO', 'SIL Competitor', 'SIL Repeat')
and t2.loanStatus in ('In Arrears','Normal')
and t2.bucketDate in ('2024-11-30')
group by 1
)
select 
new_loan_type,
case when sum(obsthirtyplus) = 0 then 0 else round(sum(thirtyplus)/sum(obsthirtyplus)*100, 3) end thirtyplus,
case when sum(obsthirtyplus) = 0 then 0 else round(sum(sixtyplus)/sum(obsthirtyplus)*100, 3) end sixtyplus,
case when sum(obsnintyplus) = 0 then 0 else round(sum(nintyplus)/sum(obsnintyplus)*100, 3) end nintyplus,
from b
group by 1
order by 1
;"""

secondpartNov24 = client.query(sq).to_dataframe(progress_bar_type='tqdm')
secondpartNov24

Job ID ef5f471f-02bb-4adc-987f-3835364a1f6a successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,new_loan_type,thirtyplus,sixtyplus,nintyplus
0,Flex,38.624,34.184,29.856
1,Quick,34.880,25.909,36.016
2,SIL Competitor,0.000,0.000,0.000
3,SIL Repeat,0.000,0.000,0.000
4,SIL ZERO,12.955,3.224,0.000
5,SIL-Instore,27.426,21.011,24.660


# 30+, 60+, 90+ for November 2024 (without Product)

In [5]:
sq = """with b as
(select 
count(distinct t2.loanAccountNumber) cnt_loan
, count(distinct case when t2.Max_current_DPD >=30 then t2.loanAccountNumber end) thirtyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >=30 then t2.loanAccountNumber end) obsthirtyplus
, count(distinct case when t2.Max_current_DPD >=60 then t2.loanAccountNumber end) sixtyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >=60 then t2.loanAccountNumber end) obssixtyplus
, count(distinct case when t2.Max_current_DPD >=90 then t2.loanAccountNumber end) nintyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >= 90 then t2.loanAccountNumber end) obsnintyplus
from `risk_credit_mis.loan_bucket_flow_report_core` t2
inner join `risk_credit_mis.loan_master_table` t1 on t2.loanAccountNumber = t1.loanAccountNumber
WHERE t1.flagDisbursement = 1
and t1.new_loan_type in ('Quick','Flex','SIL-Instore', 'SIL ZERO', 'SIL Competitor', 'SIL Repeat')
and t2.loanStatus in ('In Arrears','Normal')
and t2.bucketDate in ('2024-11-30')
)
select 
case when sum(obsthirtyplus) = 0 then 0 else round(sum(thirtyplus)/sum(obsthirtyplus)*100, 3) end as thirtyplus,
case when sum(obsthirtyplus) = 0 then 0 else round(sum(sixtyplus)/sum(obsthirtyplus)*100, 3) end as sixtyplus,
case when sum(obsnintyplus) = 0 then 0 else round(sum(nintyplus)/sum(obsnintyplus)*100, 3) end as nintyplus,
from b
order by 1
;"""

thirdpartNov24 = client.query(sq).to_dataframe(progress_bar_type='tqdm')
thirdpartNov24

Job ID ca3a02e6-b7ea-4a8b-8857-b41973997c67 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,thirtyplus,sixtyplus,nintyplus
0,28.259,21.448,26.057


In [6]:
nov24fpd30.head()

,new_loan_type,percentage_of_FPD30
0,Quick,13.853
1,SIL Competitor,0.000
2,SIL ZERO,8.960
3,Flex,0.222
4,SIL Repeat,0.000


In [7]:
secondpartNov24.head()

,new_loan_type,thirtyplus,sixtyplus,nintyplus
0,Flex,38.624,34.184,29.856
1,Quick,34.880,25.909,36.016
2,SIL Competitor,0.000,0.000,0.000
3,SIL Repeat,0.000,0.000,0.000
4,SIL ZERO,12.955,3.224,0.000


In [8]:
dfNov24 = nov24fpd30.merge(secondpartNov24, on='new_loan_type', how='left')
dfNov24

,new_loan_type,percentage_of_FPD30,thirtyplus,sixtyplus,nintyplus
0,Quick,13.853,34.880,25.909,36.016
1,SIL Competitor,0.000,0.000,0.000,0.000
2,SIL ZERO,8.960,12.955,3.224,0.000
3,Flex,0.222,38.624,34.184,29.856
4,SIL Repeat,0.000,0.000,0.000,0.000
5,SIL-Instore,8.092,27.426,21.011,24.660


In [9]:
thirdpartNov24.head()

,thirtyplus,sixtyplus,nintyplus
0,28.259,21.448,26.057


In [10]:
resultNov24 = pd.concat([dfNov24, thirdpartNov24], ignore_index=True)


In [11]:
resultNov24['new_loan_type'] = resultNov24['new_loan_type'].fillna("Portfolio")
resultNov24

,new_loan_type,percentage_of_FPD30,thirtyplus,sixtyplus,nintyplus
0,Quick,13.853,34.880,25.909,36.016
1,SIL Competitor,0.000,0.000,0.000,0.000
2,SIL ZERO,8.960,12.955,3.224,0.000
3,Flex,0.222,38.624,34.184,29.856
4,SIL Repeat,0.000,0.000,0.000,0.000
5,SIL-Instore,8.092,27.426,21.011,24.660
6,Portfolio,NaN,28.259,21.448,26.057


In [12]:
import pandas as pd
from openpyxl import load_workbook


In [13]:
# Load the existing Excel file
existing_excel_file = r'C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Collection\Anne\Jennifer Zaldivar\KRI_New_Report\KRI_Report_New\Data\20241102_KRI_Report_New.xlsx'
xls = pd.ExcelFile(existing_excel_file)

# Read each existing sheet into separate dataframes
df_january24 = pd.read_excel(xls, sheet_name='January2024')
df_february24 = pd.read_excel(xls, sheet_name='February2024')
df_march24 = pd.read_excel(xls, sheet_name='March2024')
df_april24 = pd.read_excel(xls, sheet_name='April2024')
df_may24 = pd.read_excel(xls, sheet_name='May2024')
df_june24 = pd.read_excel(xls, sheet_name='June2024')
df_july24 = pd.read_excel(xls, sheet_name='July2024')
df_aug24 = pd.read_excel(xls, sheet_name='August2024')
df_sep24 = pd.read_excel(xls, sheet_name='September2024')
df_oct24 = pd.read_excel(xls, sheet_name='October2024')



In [14]:
df_oct24

,new_loan_type,percentage_of_FPD30,thirtyplus,sixtyplus,nintyplus
0,A. Flex,0.567,39.377,35.033,30.878
1,B. Quick,12.827,35.833,26.217,37.009
2,C. SIL-Instore,7.532,24.993,18.434,22.735
3,D. Portfolio,NaN,26.583,19.667,24.595


In [15]:
# Mapping function to modify 'new_loan_type' values
def modify_loan_type(loan_type):
    mapping = {'Flex': 'A. Flex', 'SIL-Instore': 'C. SIL-Instore', 'Quick': 'B. Quick', 'SIL ZERO': 'D. SIL ZERO', 'SIL Repeat':'E. SIL Repeat', 'SIL Competitor':'F. SIL Competitor', 'Portfolio': 'G. Portfolio'}
    return mapping.get(loan_type, loan_type)  # Return original value if not found

# Apply the mapping function to the 'new_loan_type' column
resultNov24['new_loan_type'] = resultNov24['new_loan_type'].apply(modify_loan_type)
resultNov24.sort_values(by='new_loan_type')

,new_loan_type,percentage_of_FPD30,thirtyplus,sixtyplus,nintyplus
3,A. Flex,0.222,38.624,34.184,29.856
0,B. Quick,13.853,34.880,25.909,36.016
5,C. SIL-Instore,8.092,27.426,21.011,24.660
2,D. SIL ZERO,8.960,12.955,3.224,0.000
4,E. SIL Repeat,0.000,0.000,0.000,0.000
1,F. SIL Competitor,0.000,0.000,0.000,0.000
6,G. Portfolio,NaN,28.259,21.448,26.057


In [16]:
# Create a Pandas Excel writer using XlsxWriter as the engine
writer = pd.ExcelWriter(r'C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Collection\Anne\Jennifer Zaldivar\KRI_New_Report\KRI_Report_New\Data\20241202_KRI_Report_New.xlsx', engine='xlsxwriter')

# Sort each DataFrame by the first column and write to a separate worksheet, ignoring the index
df_january24.sort_values(by=df_january24.columns[0]).to_excel(writer, sheet_name='January2024', index=False)
df_february24.sort_values(by=df_february24.columns[0]).to_excel(writer, sheet_name='February2024', index=False)
df_march24.sort_values(by=df_march24.columns[0]).to_excel(writer, sheet_name='March2024', index=False)
df_april24.sort_values(by=df_april24.columns[0]).to_excel(writer, sheet_name='April2024', index=False)
df_may24.sort_values(by=df_may24.columns[0]).to_excel(writer, sheet_name='May2024', index=False)
df_june24.sort_values(by=df_june24.columns[0]).to_excel(writer, sheet_name='June2024', index=False)
df_july24.sort_values(by=df_july24.columns[0]).to_excel(writer, sheet_name='July2024', index=False)
df_aug24.sort_values(by=df_aug24.columns[0]).to_excel(writer, sheet_name='August2024', index=False)
df_sep24.sort_values(by=df_sep24.columns[0]).to_excel(writer, sheet_name='September2024', index=False)
df_oct24.sort_values(by=df_oct24.columns[0]).to_excel(writer, sheet_name='Octboer2024', index = False)
resultNov24.sort_values(by=resultNov24.columns[0]).to_excel(writer, sheet_name='November2024', index=False)

# Close the Pandas Excel writer and output the Excel file
writer.close()